In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta

%matplotlib inline
pd.set_option('display.max_columns',None) #设置显示所有的列，而不是显示。。。

In [2]:
import re

In [3]:
def readData(number=1):
    if number == 1:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_1.txt',sep='\t')
        return df
    elif number == 2:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_2.txt',sep='\t')
        return df
    elif number == 3:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_3.txt',sep='\t')
        return df
    elif number == 4:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_4.txt',sep='\t')
        return df
    elif number == 5:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_5.txt',sep='\t')
        return df
    elif number == 6:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_6.txt',sep='\t')
        return df
    elif number == 7:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_7.txt',sep='\t')
        return df
    elif number == 8:
        df = pd.read_csv('D:\lss\ccfData\HaiKou\dwv_order_make_haikou_8.txt',sep='\t')
        return df

In [4]:
# 将文件添加到dfs列表中
dfs = []
for i in range(1,9):
    dfs.append(readData(i))

for everyDf in dfs:
    everyDf.columns=['order_id',
       'product_id', 'city_id',
       'district', 'county',
       'type', 'dwv_order_make_haikou_1.combo_type',
       'traffic_type',
       'passenger_count',
       'driver_product_id',
       'start_dest_distance',
       'arrive_time',
       'departure_time',
       'pre_total_fee',
       'normal_time',
       'bubble_trace_id',
       'product_1level',
       'dest_lng', 'dest_lat',
       'starting_lng',
       'starting_lat', 'year',
       'month', 'day']

In [5]:
# 将多个表连接在一起，进行分析。
data = pd.concat(dfs,ignore_index=True)
# data = df.copy()

In [6]:
data.columns
# data.dtypes

Index(['order_id', 'product_id', 'city_id', 'district', 'county', 'type',
       'dwv_order_make_haikou_1.combo_type', 'traffic_type', 'passenger_count',
       'driver_product_id', 'start_dest_distance', 'arrive_time',
       'departure_time', 'pre_total_fee', 'normal_time', 'bubble_trace_id',
       'product_1level', 'dest_lng', 'dest_lat', 'starting_lng',
       'starting_lat', 'year', 'month', 'day'],
      dtype='object')

In [7]:
# 将列名提取出来方便使用
order_id = data.columns[0]
product_id = data.columns[1]    #产品线id
city_id = data.columns[2]
district = data.columns[3]
county = data.columns[4]        #记录县区id
type = data.columns[5]          #订单时效
combo_type = data.columns[6]
traffic_type = data.columns[7]
passenger_count = data.columns[8]
driver_product_id = data.columns[9]   #司机子产品线
start_dest_distance = data.columns[10]   #预估路面距离
arrive_time = data.columns[11]      #到达时间
departure_time = data.columns[12]   #出发时间
pre_total_fee = data.columns[13]    #预估价格
normal_time  = data.columns[14]     #时长
bubble_trace_id = data.columns[15]
product_1level = data.columns[16]   #业务类型
dest_lng  = data.columns[17]
dest_lat  = data.columns[18]
starting_lng = data.columns[19]
starting_lat = data.columns[20]
year = data.columns[21]   
month = data.columns[22]
day = data.columns[23]

可以看出来，海口**滴滴快车**出行的起步价为9元，平均消费价格为20元

In [8]:
data[((data[pre_total_fee]>0)&(data[product_id]==3))][pre_total_fee].describe() #对订单预测价格分析

count    1.400163e+07
mean     2.029472e+01
std      5.937229e+01
min      9.000000e+00
25%      1.170000e+01
50%      1.620000e+01
75%      2.240000e+01
max      1.779030e+04
Name: pre_total_fee, dtype: float64

海口**滴滴专车**的起步价为11元，订单的平均价格为63元；
快车与专车的使用比例为：1333157:14957大约为**89:1**的比例；

In [9]:
data[((data[pre_total_fee]>0) & (data[product_id]==1))][pre_total_fee].describe() #对订单预测价格分析

count    156827.000000
mean         63.385869
std         152.588491
min          11.000000
25%          26.600000
50%          50.000000
75%          84.400000
max       15444.700000
Name: pre_total_fee, dtype: float64

可以看出product_1level和product_id的数据完全相同，可以删除product_1level列的数据

In [10]:
data[data[product_1level]==data[product_id]].shape[0]

14158769

In [1]:
data.shape[0]

NameError: name 'data' is not defined

In [12]:
data = data.drop(columns=product_1level)  #删除表中多余的列

In [13]:
data.shape[1]

23

In [14]:
# 统计缺失值的数量和百分比
total = data.isnull().sum().sort_values(ascending=False)
precent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,precent],axis=1,keys=['Total','Percent'])
missing_data.head()

,Total,Percent
normal_time,1974224,0.139421
bubble_trace_id,199129,0.014063
day,0,0.000000
driver_product_id,0,0.000000
product_id,0,0.000000


对于**距离小于200米，且时长为NULL**的数据有必要考虑进行丢弃

In [15]:
data[(data[start_dest_distance]<200) & (np.isnan(data[normal_time]))].shape[0]  #在判断是否是NaN的时候调用了numpy的isnan

3098

In [16]:
# 将满足上述条件的每一条数据进行删除
data.drop(index=data[(data[start_dest_distance]<200) & (np.isnan(data[normal_time]))].index,inplace=True)

In [17]:
#通过gruopby对出行次数和距离情况进行分析
def dealGoOutNubAndDis(data):
    goOutNumberAndDistance = data.groupby([month,day])[start_dest_distance].agg(['count','sum','mean'])
    reData = goOutNumberAndDistance.reset_index()
    reData.to_csv('D:\lss\ccfData\HaiKouDeal\goOutNumberAndDistance.csv',index=False)
# dealGoOutNubAndDis(data)   
# goOutNumberAndDistance = data.groupby([month,day])[start_dest_distance].agg(['count','sum','mean'])

通过groups可以获取到所分组中每一项的索引值，按天来获取经纬度，后面分析经纬度的变化，来进一步分析出发地和目的地的变化

In [18]:
def dealStart2dest(data):
    monthAndDay = data.groupby([month,day])[month,day,start_dest_distance,dest_lng,dest_lat].groups
    for i in list(monthAndDay.keys()):
        (data.groupby([month,day])[month,day,start_dest_distance,starting_lng,starting_lat,dest_lng,dest_lat].get_group(i)).to_csv('D:\lss\ccfData\HaiKouDeal\StartToDest\\'+str(i)+'.csv')
# dealStart2dest(data)
# dest5_3=data.groupby([month,day])[month,day,start_dest_distance,dest_lng,dest_lat].get_group((5,3))

In [19]:
# 预约单数：266264
# 没有到达时间的有：101444
print('预约订单数:',data[data[type]==1].shape[0])
print('进行了预约，没有到达时间的订单数:',data[(data[type]==1) & (data[arrive_time]=='0000-00-00 00:00:00')].shape[0])

预约订单数: 266264
进行了预约，没有到达时间的订单数: 101444


In [20]:
# 获取到原始数据中的实时订单的相关信息
# timeData = data[data[type]==0][[month,day,departure_time]]
# timeData = pd.DataFrame(timeData)

In [21]:
# 获取订单填写时的时间在在几点
# timeData.loc[:,'transTime'] = pd.to_datetime(timeData[departure_time],
#                                              format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%m-%d %H'})

In [22]:
# 对表中的month和day进行修正，以出发时间为准
# timeData.loc[:,month] = pd.to_datetime(timeData[departure_time],
#                                              format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%m'})
# timeData.loc[:,day] = pd.to_datetime(timeData[departure_time],
#                                              format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%d'})

In [23]:
# 统计每一天每个小时内的订单数量
# timeData.groupby([month,day,'transTime']).agg('count').to_csv('D:\lss\ccfData\HaiKouDeal\everyHourCount.csv')

In [24]:
def dealEveryHourCount(data):
    # 获取到原始数据中的实时订单的相关信息
    timeData = data[data[type]==0][[month,day,departure_time]]
    timeData = pd.DataFrame(timeData)
    # 获取订单填写时的时间在在几点
    timeData.loc[:,'transTime'] = pd.to_datetime(timeData[departure_time],
                                             format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%m-%d %H'})
    # 对表中的month和day进行修正，以出发时间为准
    timeData.loc[:,month] = pd.to_datetime(timeData[departure_time],
                                             format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%m'})
    timeData.loc[:,day] = pd.to_datetime(timeData[departure_time],
                                             format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%d'})
    # 统计每一天每个小时内的订单数量
    timeData.groupby([month,day,'transTime']).agg('count').to_csv('D:\lss\ccfData\HaiKouDeal\everyHourCount.csv')
# dealEveryHourCount(data)

In [25]:
palleData = data[[type,start_dest_distance,arrive_time,departure_time,pre_total_fee,normal_time,month,day]]
palleData.head(5)

,type,start_dest_distance,arrive_time,departure_time,pre_total_fee,normal_time,month,day
0,0,4361,2017-05-19 01:09:12,2017-05-19 01:05:19,13.0,11.0,5,19
1,0,5756,2017-05-19 02:08:13,2017-05-19 02:04:38,17.0,10.0,5,19
2,0,2828,2017-05-19 02:35:16,2017-05-19 02:33:07,9.0,7.0,5,19
3,0,4854,2017-05-19 07:49:19,2017-05-19 07:47:21,14.0,11.0,5,19
4,0,4478,2017-05-19 08:13:32,2017-05-19 08:10:58,12.0,12.0,5,19


In [26]:
# def dealStart2dest(data):
#     monthAndDay = data.groupby([month,day])[month,day,start_dest_distance,dest_lng,dest_lat].groups
#     for i in list(monthAndDay.keys()):
#         (data.groupby([month,day])[month,day,start_dest_distance,starting_lng,starting_lat,dest_lng,dest_lat].get_group(i)).to_csv('D:\lss\ccfData\HaiKouDeal\StartToDest\\'+str(i)+'.csv')
palleData.drop(index=palleData[(palleData[arrive_time]=='0000-00-00 00:00:00')].index,inplace=True)

D:\Program\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [27]:
palleData.loc[:,arrive_time] = pd.to_datetime(palleData[arrive_time],
                                              format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%Y-%m-%d %H:%M'})

D:\Program\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [39]:
palleData.loc[:,'arrive_temp'] = pd.to_datetime(palleData[arrive_time],
                                              format='%Y-%m-%d %H:%M').apply(datetime.strftime,**{'format':'(%m, %d)'})

In [29]:
palleData.loc[:,departure_time] = pd.to_datetime(palleData[departure_time],
                                              format='%Y-%m-%d %H:%M:%S').apply(datetime.strftime,**{'format':'%Y-%m-%d %H:%M'})

D:\Program\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [40]:
palleData.loc[:,'departure_temp'] = pd.to_datetime(palleData[departure_time],
                                              format='%Y-%m-%d %H:%M').apply(datetime.strftime,**{'format':'(%m, %d)'})

In [31]:
# palleData.groupby([month,day])[type,start_dest_distance,arrive_time,departure_time,pre_total_fee,normal_time].get_group((5, 1)).to_csv('D:\lss\ccfData\HaiKouDeal\palleData\\'+'(5, 1)'+'.csv',index=0)

In [41]:
palleData = palleData.dropna(axis=0)
palleData['arrive_temp']=palleData['arrive_temp'].map(lambda x : ', '.join(re.findall(r'[1-9]+[0-9]*',x))).map(lambda x: '(' + x + ')')
palleData['departure_temp']=palleData['departure_temp'].map(lambda x : ', '.join(re.findall(r'[1-9]+[0-9]*',x))).map(lambda x: '(' + x + ')')

In [42]:
def getPalleData(palleData):
    
    dateList = list(palleData.groupby([month,day]).groups.keys())
    for i in dateList:
        todayData = palleData.groupby([month,day])[type,start_dest_distance,arrive_time,departure_time,
                                       pre_total_fee,normal_time,'arrive_temp','departure_temp'].get_group(i)

        print(todayData)
        todayData = todayData.drop(index=todayData[(todayData[start_dest_distance]>28000) | (todayData[normal_time]<3)| (todayData[pre_total_fee]>100) | (todayData[normal_time]>100)].index)
        todayData = todayData.drop(index=todayData[todayData['arrive_temp']!=str(i)].index)
        todayData = todayData.drop(index=todayData[todayData['departure_temp']!=str(i)].index)
        todayData = pd.DataFrame(todayData)
        todayData[[type,start_dest_distance,arrive_time,departure_time,pre_total_fee,
                      normal_time]].to_csv('D:\lss\ccfData\HaiKouDeal\palleData\\' + str(i) + '.csv',index=0)
getPalleData(palleData)

        type  start_dest_distance       arrive_time    departure_time  \
422451     1                25650  2017-05-01 04:32  2017-05-01 05:20   
422452     0                 5326  2017-04-30 23:57  2017-04-30 23:53   
422453     0                 6693  2017-05-01 00:26  2017-05-01 00:24   
422454     0                 5345  2017-05-01 00:11  2017-05-01 00:10   
422455     0                24022  2017-05-01 00:28  2017-05-01 00:27   
422456     0                10149  2017-05-01 01:30  2017-05-01 01:24   
422457     0                 6860  2017-05-01 02:07  2017-05-01 02:02   
422458     0                 5002  2017-05-01 02:43  2017-05-01 02:40   
422460     0                18513  2017-05-01 07:32  2017-05-01 07:23   
422461     0                 4737  2017-05-01 08:03  2017-05-01 07:57   
422462     0                17703  2017-05-01 08:36  2017-05-01 08:31   
422463     0                 3713  2017-05-01 09:09  2017-05-01 09:05   
422464     0                 4909  2017-05-01 09:17

        type  start_dest_distance       arrive_time    departure_time  \
770761     0                 9121  2017-05-01 23:46  2017-05-01 23:42   
770762     0                 2309  2017-05-02 01:14  2017-05-02 01:11   
770764     0                 7730  2017-05-02 06:51  2017-05-02 06:47   
770765     0                11758  2017-05-02 08:01  2017-05-02 07:51   
770768     0                 5793  2017-05-02 09:18  2017-05-02 09:15   
770769     0                 7740  2017-05-02 09:27  2017-05-02 09:23   
770770     0                 3227  2017-05-02 09:51  2017-05-02 09:49   
770772     0                 5052  2017-05-02 10:38  2017-05-02 10:32   
770774     0                 3403  2017-05-02 10:37  2017-05-02 10:34   
770775     0                 3102  2017-05-02 10:42  2017-05-02 10:37   
770776     0                 1164  2017-05-02 10:51  2017-05-02 10:45   
770777     0                12598  2017-05-02 11:38  2017-05-02 11:34   
770778     0                 8163  2017-05-02 11:47

        type  start_dest_distance       arrive_time    departure_time  \
367390     0                11131  2017-05-02 23:45  2017-05-02 23:39   
367391     0                 5969  2017-05-02 23:57  2017-05-02 23:52   
367392     0                 8594  2017-05-03 00:30  2017-05-03 00:28   
367394     0                 3419  2017-05-03 02:34  2017-05-03 02:31   
367396     0                  880  2017-05-03 07:45  2017-05-03 07:41   
367397     0                 4713  2017-05-03 08:02  2017-05-03 07:55   
367398     0                 5657  2017-05-03 08:24  2017-05-03 08:21   
367399     0                 4424  2017-05-03 08:33  2017-05-03 08:27   
367400     0                 3462  2017-05-03 08:45  2017-05-03 08:39   
367401     0                 2641  2017-05-03 08:55  2017-05-03 08:48   
367402     0                 5620  2017-05-03 08:56  2017-05-03 08:48   
367403     0                 8259  2017-05-03 08:54  2017-05-03 08:51   
367405     0                 5563  2017-05-03 09:56

         type  start_dest_distance       arrive_time    departure_time  \
1296113     1                15795  2017-05-04 06:44  2017-05-04 06:50   
1296114     0                18749  2017-05-04 00:05  2017-05-04 00:03   
1296115     0                 2651  2017-05-04 00:48  2017-05-04 00:45   
1296116     0                 6662  2017-05-04 01:53  2017-05-04 01:49   
1296117     0                 4382  2017-05-04 03:29  2017-05-04 03:24   
1296118     0                 9420  2017-05-04 06:24  2017-05-04 06:23   
1296119     0                 8385  2017-05-04 06:46  2017-05-04 06:44   
1296120     0                16531  2017-05-04 07:09  2017-05-04 07:07   
1296122     0                 6488  2017-05-04 08:02  2017-05-04 08:02   
1296123     0                 3107  2017-05-04 08:27  2017-05-04 08:21   
1296124     0                 3821  2017-05-04 08:32  2017-05-04 08:28   
1296125     0                 5309  2017-05-04 08:44  2017-05-04 08:35   
1296126     0                 5739  20

        type  start_dest_distance       arrive_time    departure_time  \
304840     0                 3056  2017-05-05 01:48  2017-05-05 01:44   
304841     0                 4313  2017-05-05 07:06  2017-05-05 07:03   
304842     0                 2582  2017-05-05 07:44  2017-05-05 07:43   
304843     0                 2564  2017-05-05 08:13  2017-05-05 08:07   
304844     0                 6652  2017-05-05 08:35  2017-05-05 08:32   
304845     0                11940  2017-05-05 09:11  2017-05-05 08:59   
304846     0                 8472  2017-05-05 09:23  2017-05-05 09:14   
304848     0                 5863  2017-05-05 09:54  2017-05-05 09:52   
304849     0                 7106  2017-05-05 10:16  2017-05-05 10:11   
304850     0                 5837  2017-05-05 10:31  2017-05-05 10:31   
304851     0                 3094  2017-05-05 10:33  2017-05-05 10:31   
304852     0                 3706  2017-05-05 10:48  2017-05-05 10:46   
304854     0                 3247  2017-05-05 11:47

...

          type  start_dest_distance       arrive_time    departure_time  \
11997984     1                12666  2017-10-27 21:21  2017-10-27 21:20   
11997986     1                20859  2017-10-27 05:13  2017-10-27 05:20   
11997987     0                13762  2017-10-27 00:31  2017-10-27 00:30   
11997988     0                 4972  2017-10-27 01:02  2017-10-27 01:01   
11997989     0                 5741  2017-10-27 01:17  2017-10-27 01:13   
11997990     0                25670  2017-10-27 06:02  2017-10-27 06:01   
11997991     0                 3422  2017-10-27 07:08  2017-10-27 07:07   
11997992     0                 6146  2017-10-27 07:27  2017-10-27 07:25   
11997993     0                10031  2017-10-27 07:26  2017-10-27 07:21   
11997994     0                 4745  2017-10-27 07:47  2017-10-27 07:40   
11997995     1                17134  2017-10-27 08:00  2017-10-27 08:10   
11997996     0                 6542  2017-10-27 08:08  2017-10-27 08:02   
11997997     0           

          type  start_dest_distance       arrive_time    departure_time  \
12899782     0                20752  2017-10-28 22:06  2017-10-28 21:59   
12899783     1                17796  2017-10-28 04:51  2017-10-28 04:40   
12899785     0                10422  2017-10-28 01:20  2017-10-28 01:17   
12899786     0                 3936  2017-10-28 01:12  2017-10-28 01:08   
12899787     0                 7094  2017-10-28 02:10  2017-10-28 02:07   
12899788     0                 4735  2017-10-28 02:16  2017-10-28 02:11   
12899789     0                 5992  2017-10-28 07:34  2017-10-28 07:27   
12899790     0                 4784  2017-10-28 08:31  2017-10-28 08:27   
12899791     0                 2088  2017-10-28 08:38  2017-10-28 08:27   
12899792     0                 9095  2017-10-28 08:23  2017-10-28 08:19   
12899793     0                 5864  2017-10-28 08:31  2017-10-28 08:20   
12899794     0                 3121  2017-10-28 09:00  2017-10-28 08:56   
12899795     0           

          type  start_dest_distance       arrive_time    departure_time  \
10694163     0                 8037  2017-10-28 23:58  2017-10-28 23:52   
10694164     0                 4464  2017-10-29 00:05  2017-10-28 23:59   
10694165     0                 4608  2017-10-29 00:19  2017-10-29 00:18   
10694166     0                 3365  2017-10-29 01:57  2017-10-29 01:48   
10694167     0                 6506  2017-10-29 01:53  2017-10-29 01:50   
10694168     0                 7520  2017-10-29 02:54  2017-10-29 02:47   
10694169     0                11669  2017-10-29 01:29  2017-10-29 01:26   
10694170     0                 6500  2017-10-29 03:49  2017-10-29 03:44   
10694171     0                 7152  2017-10-29 06:50  2017-10-29 06:47   
10694172     0                 6599  2017-10-29 06:46  2017-10-29 06:43   
10694173     0                 9996  2017-10-29 08:27  2017-10-29 08:25   
10694175     0                 5318  2017-10-29 09:08  2017-10-29 09:06   
10694176     0           

          type  start_dest_distance       arrive_time    departure_time  \
10621024     0                 3396  2017-10-30 00:03  2017-10-30 00:00   
10621025     0                 4896  2017-10-30 00:59  2017-10-30 00:55   
10621026     0                 3994  2017-10-30 01:59  2017-10-30 01:55   
10621027     0                 6843  2017-10-30 05:14  2017-10-30 05:10   
10621030     0                 6186  2017-10-30 07:29  2017-10-30 07:25   
10621032     0                 6487  2017-10-30 07:53  2017-10-30 07:50   
10621033     0                 2195  2017-10-30 08:20  2017-10-30 08:16   
10621034     0                 2990  2017-10-30 08:38  2017-10-30 08:36   
10621035     0                 3210  2017-10-30 08:29  2017-10-30 08:22   
10621036     0                 3708  2017-10-30 09:02  2017-10-30 09:00   
10621037     0                 2790  2017-10-30 09:26  2017-10-30 09:26   
10621039     0                 9805  2017-10-30 10:02  2017-10-30 09:58   
10621041     0           

          type  start_dest_distance       arrive_time    departure_time  \
10469616     0                 7335  2017-10-31 21:19  2017-10-31 21:14   
10469618     0                 7505  2017-10-30 23:57  2017-10-30 23:55   
10469619     0                 1929  2017-10-30 23:59  2017-10-30 23:56   
10469620     0                 6511  2017-10-31 02:46  2017-10-31 02:41   
10469621     0                 6828  2017-10-31 04:39  2017-10-31 04:34   
10469622     0                14825  2017-10-31 07:17  2017-10-31 07:12   
10469623     0                 8444  2017-10-31 07:44  2017-10-31 07:41   
10469625     0                 3921  2017-10-31 08:16  2017-10-31 08:13   
10469627     0                 8617  2017-10-31 08:54  2017-10-31 08:50   
10469628     0                10089  2017-10-31 08:43  2017-10-31 08:38   
10469629     0                 4529  2017-10-31 08:55  2017-10-31 08:54   
10469631     0                12568  2017-10-31 09:39  2017-10-31 09:33   
10469633     0           